In [1]:
import pandas as pd
import boto3
import os
from io import StringIO

c:\Users\hamin\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Where to get the key

Contact Minh and put the file in the root directory

In [7]:
# Get the current working directory
current_dir = os.getcwd()
# Move up one level from the current directory
parent_dir = os.path.dirname(os.path.dirname(current_dir))

# Link to data dir
data_dir = os.path.join(parent_dir, 'data')

# key_df
key_df = pd.read_csv(os.path.join(parent_dir, 'british-airway-user.csv'), index_col=None)


## Read data 

Read the latest file in the bucket


In [4]:
# Initialize a session using Amazon S3
s3_client = boto3.client('s3', aws_access_key_id=key_df['Access key ID'][0], aws_secret_access_key=key_df['Secret access key'][0])

# Name of the S3 bucket
bucket_name = 'new-british-airline'

# Function to get the two most recent CSV files
def get_recent_csv_files(bucket_name, num_files=2):
    csv_files = []
    response = s3_client.list_objects_v2(Bucket=bucket_name)
    for obj in response.get('Contents', []):
        if obj['Key'].endswith('.csv'):
            csv_files.append({'Key': obj['Key'], 'LastModified': obj['LastModified']})
    
    # Sort the files by last modified date in descending order and get the top 'num_files' entries
    recent_csv_files = sorted(csv_files, key=lambda x: x['LastModified'], reverse=True)[:num_files]
    return [file['Key'] for file in recent_csv_files]

# Function to read a CSV file from S3 into a DataFrame
def read_csv_to_df(bucket_name, file_key):
    csv_obj = s3_client.get_object(Bucket=bucket_name, Key=file_key)
    body = csv_obj['Body']
    csv_string = body.read().decode('utf-8')
    df = pd.read_csv(StringIO(csv_string))
    return df

# Get the two most recent CSV files

recent_csv_files = get_recent_csv_files(bucket_name)

# You can now loop through the file keys or handle them individually
# Example: Read the files into DataFrames
dataframes = [read_csv_to_df(bucket_name, file_key) for file_key in recent_csv_files]

df= dataframes[0]
previous_df= dataframes[1]

In [5]:
df.head()

,id,date_review,day_review,month_review,month_review_num,year_review,verified,verified.1,name,month_fly,...,seat_comfort,cabit_serv,food,ground_service,wifi,money_value,score,experience,recommended,review
0,0,2024-04-03,3,April,4,2024,True,True,Mike Flynn,March,...,3.0,5.0,3.0,4.0,NaN,3,3.75,Fair,True,Check in fast and friendly. Security a breeze....
1,1,2024-04-02,2,April,4,2024,True,True,David Williams,March,...,5.0,1.0,1.0,5.0,1.0,3,2.60,Fair,False,Flew business class from Doha to London on 31s...
2,2,2024-04-02,2,April,4,2024,True,True,E Dodson,March,...,3.0,1.0,1.0,2.0,1.0,1,1.60,Poor,False,This is the first time I have seen the new Clu...
3,3,2024-03-28,28,March,3,2024,True,True,C Down,March,...,4.0,4.0,5.0,4.0,NaN,4,4.25,Good,True,Starting off at Heathrow Terminal 3. Check in ...
4,4,2024-03-28,28,March,3,2024,False,False,Robert Walls,March,...,4.0,3.0,NaN,2.0,NaN,1,3.00,Poor,False,We have flown this route with EasyJet regularl...


In [9]:
df.to_csv(os.path.join(data_dir, 'data.csv'), index = False)